In [ ]:
import pandas as pd
import numpy as np
import clickhouse_connect
from Config import passwd
from collections import defaultdict

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_colwidth', None)

db = clickhouse_connect.get_client(host='localhost',username='admin',password=passwd,compression=True)
"""
08:45 開盤（期貨）	00:45 UTC	台指期開始報價，根據國際盤、ADR 預期方向
09:00 開盤（現貨）	01:00 UTC	現貨開盤，出現真實流動性與方向確認
09:30~11:00	01:30~03:00 UTC	主動資金進場區，早盤高頻策略與波動放大期
11:00~12:30	03:00~04:30 UTC	中盤盤整，高低點初步確認，可做 mean-revert
13:00~13:30	05:00~05:30 UTC	收盤前調倉、現貨權重股尾盤拉抬壓低
~15:00 夜盤	07:00 UTC	台指夜盤開，根據歐美消息重新定價

Estern Time
08:00 美期活躍啟動	12:00 UTC	    ES/NQ 電子盤流動性上升，對亞洲夜盤有牽引力
08:30~09:30	       12:30~13:30 UTC	重磅 macro 數據發布(CPI、NFP)時段
09:30 開盤（現貨）	13:30 UTC	    SPY、QQQ 現貨開市，量能爆發，方向確認
09:30~11:00	       13:30~15:00 UTC	趨勢發展期，對亞股夜盤牽引力最大
14:00~16:00	       18:00~20:00 UTC	ETF 尾盤 TWAP、大宗基金再平衡、期權對沖
16:00 收盤	       20:00 UTC	    US 現貨結束，期貨流動性驟降，進入夜間盤整段

00:00 ── 美期預測台期開盤
00:45 ── 台指期開盤
01:00 ── 台灣現貨開盤
   │
03:00 ── 亞洲午盤前（台股波動段結束）
05:30 ── 台股現貨收盤（期貨續盤）
07:00 ── 台指夜盤開啟，等待美盤引導
   │
12:00 ── ES/NQ 開始活躍（影響台指夜盤）
13:30 ── SPY 現貨開盤 → 真實方向
15:00 ── 美股主升段（影響夜盤大）
18:00 ── TWAP 開始 → 尾盤套利出現
20:00 ── 美股收盤


Domestic:
Separate analytic by session. Asia Session: 8:46~13:45, NY Session: 21:31~4:00 
consider rest of time low volume (all of them has statistic different)

Oversea:
Has "significant" volume at NY session. (skip rest of time or separate the analytics)
do not close app during NY session
"""

query_TX="""
    SELECT *
    FROM orderflowDM
    WHERE symbol = 'TX00'
    AND time>=today()
    ORDER BY time 
"""
query_MTX="""
    SELECT time, open, high, low, close, volume, delta, trades_delta, price_map
    FROM orderflowDM
    WHERE symbol = 'MTX00'
    ORDER BY time 
"""
symbol="'MTX00'"
q = f"""
    SELECT time,open,high,low,close,volume
    FROM orderflowDM
    WHERE symbol = {symbol}
    ORDER BY time
"""
TX_df = db.query_np(q)
TX_df
# MTX_df:pd.DataFrame = db.query_df(query_MTX).set_index('time')

# len(TX_df.index), len(MTX_df.index)
# TX_df.to_csv('txfp.csv')


array([('2025-05-13T07:01:00', 21353., 21365., 21345., 21349., 523),
       ('2025-05-13T07:02:00', 21349., 21353., 21342., 21344., 219),
       ('2025-05-13T07:03:00', 21346., 21358., 21343., 21348., 156), ...,
       ('2025-06-24T07:58:00', 21972., 21972., 21968., 21972., 105),
       ('2025-06-24T07:59:00', 21973., 21979., 21973., 21978., 108),
       ('2025-06-24T08:00:00', 21978., 21980., 21976., 21978.,  59)],
      shape=(33049,), dtype=[('time', '<M8[s]'), ('open', '<f4'), ('high', '<f4'), ('low', '<f4'), ('close', '<f4'), ('volume', '<u4')])

In [ ]:
insert_row=[]

data=[pd.Timestamp('2025-06-11 07:01:00-0500', tz='America/Chicago'),'NQ0000', 21931.0, 21935.0, 21928.0, 21934.75, 180, (39, 0, 38), 13, []]
data[-1]=[]
insert_row.append(tuple(data))
insert_row
# db.insert('orderflowOS',insert_row)

In [35]:
fp=pd.read_csv("TX_Tick.csv").set_index('Time')
fp['delta'] = abs(fp['aggBuy']-fp['aggSell'])
condition = df['close'] > df['open']

condition_met_indices = df[condition].index
condition_met_indices

n = 5
future_closes = np.column_stack([
    df['close'].shift(-i) for i in range(1, n + 1)
])

current_close = df['close'].values.reshape(-1, 1)
future_returns = (future_closes - current_close) / current_close

# Avg, max, min return, volatility
df['avg_return'] = np.nanmean(future_returns, axis=1)
df['max_return'] = np.nanmax(future_returns, axis=1)
df['min_return'] = np.nanmin(future_returns, axis=1)
df['volatility'] = np.nanstd(future_returns, axis=1)

def max_drawdown(series):
    peak = series[0]
    drawdowns = [(price - peak) / peak if price <= peak else (0 if (peak := price) else 0) for price in series]
    return min(drawdowns)

df['max_drawdown'] = pd.DataFrame(future_closes).apply(max_drawdown, axis=1)


DatetimeIndex(['2025-04-25 15:01:00+08:00', '2025-04-25 15:02:00+08:00',
               '2025-04-25 15:03:00+08:00', '2025-04-25 15:04:00+08:00',
               '2025-04-25 15:08:00+08:00', '2025-04-25 15:09:00+08:00',
               '2025-04-25 15:16:00+08:00', '2025-04-25 15:17:00+08:00',
               '2025-04-25 15:21:00+08:00', '2025-04-25 15:22:00+08:00',
               ...
               '2025-04-29 19:10:00+08:00', '2025-04-29 19:11:00+08:00',
               '2025-04-29 19:13:00+08:00', '2025-04-29 19:14:00+08:00',
               '2025-04-29 19:15:00+08:00', '2025-04-29 19:17:00+08:00',
               '2025-04-29 19:19:00+08:00', '2025-04-29 19:23:00+08:00',
               '2025-04-29 19:25:00+08:00', '2025-04-29 19:29:00+08:00'],
              dtype='datetime64[s, Asia/Taipei]', name='time', length=1032, freq=None)

In [85]:
last=db.query('select time from orderflowOS order by time desc limit 1').result_rows[0][0]
# ts=pd.to_datetime('20250501',format='%Y%m%d').replace(hour=5).tz_localize('Asia/Taipei')
# data=[[ts,'tmp',0,0,0,0,0,0,0,{0:(0,0,0)}]]
# db.insert('orderflowOS',data)
pd.Timestamp(last)

Timestamp('2025-04-30 16:00:00-0500', tz='America/Chicago')